This is a Knowledge Graph Question Answering System based on the paper:  
S. Aghaei, E. Raad and A. Fensel, "Question Answering Over Knowledge Graphs: A Case Study in Tourism," in IEEE Access, vol. 10, pp. 69788-69801, 2022, doi: 10.1109/ACCESS.2022.31871

There are some deviations.  Attempts were made to note this, but the knowledge graph (KG) for this project is a couple orders of magnitude smaller (with about 400 facts/triples).  Due to time contraints some corners might have been cut.

I will discuss this in a few sections.

# Zoning Question Answering

This is just a simple example showing this work.

In [1]:
%%time
# first time this is run Stanza will download models, this will take about 1 minute.
import semantic_parsing
sem_par = semantic_parsing.SemanticParsingClass()

2022-12-02 18:37:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-02 18:37:51 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-02 18:37:51 INFO: Use device: cpu
2022-12-02 18:37:51 INFO: Loading: tokenize
2022-12-02 18:37:51 INFO: Loading: pos
2022-12-02 18:37:52 INFO: Loading: lemma
2022-12-02 18:37:52 INFO: Loading: depparse
2022-12-02 18:37:52 INFO: Done loading processors!


CPU times: user 3.89 s, sys: 1.43 s, total: 5.32 s
Wall time: 40.9 s


Below are a few random questions you could ask, rerun the cell to generate new ones or change the `n` value to get more/fewer questions.  

These are from the templates in `generate_template.py` that execute SPARQL to get the answers.  The question corpus has 900 questions in all (162 questions were excluded because of bad answers those would take time to fix them, so the total number of questions is 1062 before exclusion).

In [2]:
%%time
itr = semantic_parsing.get_random_questions_answers(sem_par.generate_filtered_corpus(), n=5)
for qa in itr:
    print(f"Question: {qa['question']}")
    if qa['answer'] is True:
        print(f"Answer: Yes")
    else:
        print(f"Answer: {qa['answer']}")

Question: What is the maximum density in the R3b zoning district?
Answer: ['12 [du/acr_u]']
Question: I would like to build quarries.  Which zoning districts permits this use?
Answer: ['FI3']
Question: I would like to build carpet manufacturing.  Which zoning districts permits this use?
Answer: ['FI2', 'FI3']
Question: I would like to build salt works.  Which zoning districts permits this use?
Answer: ['FI3']
Question: Are mortuary and funeral homes permitted in a C1 zoning district?
Answer: Yes
CPU times: user 1.42 s, sys: 11.6 ms, total: 1.43 s
Wall time: 1.43 s


Below is the Zoning KGQAS system working.

Copy the above text.  Or try to use some variation on a question.

Example questions:
* A question could be "Are quarries allowed?" is in the dataset, but mining is not so "Is mining allowed?" would be a similar use (the answer should be "Yes").  (It correctly identifies that quarries and mining are similar words.)

* The International Zoning Code uses "financial services", so asking "Is a bank allowed?" a similar use.  (the answer should be "Yes").

I asked a question, which off the top of my head I think have a No answer: 
* "Is a trailer park allowed?"  It answers "Yes," it is matching "publicly owned and operated parks".  
* Then, I ask "Which zoning districts allow trailer parks?", again it answers "Yes," matching the same Question Classification.  It matched the same question An answer that would I could beleive would be "R1" (or some other R District).  I guess I'll have to accept that Zoing KGQAS is a people pleaser.   Kidding aside, it has overfitted because only questions with Yes answers were sent to it in training.
* I ask "Is a marijuana dispensery allowed?", it matches that use as "A2" which is an Agricultural zoning district.  It gives the correct answer of "No", but it doesn't know that it should be "No".
* I ask "I would like to build a marijuana dispensery.  Which zoning districts permits this use?"  It correctly gives an empty list [].

This is thought provoking, but I'd like more experience with this before I suggest ways to fix this.

In [3]:
%%time
# question = "What is the minimum lot depth in the C3 zoning district?"   # Answer: 100 [ft_i]
# question = "Are quarries allowed?"  # A: Yes
question = "Is mining allowed?"       # A: Yes
# question = "Is a bank allowed?"     # A: Yes
# question = "Is a trailer park allowed?"   # A: Yes   (Seems incorrect)
# question = "Which zoning districts allow trailer parks?"  # A: Yes   (Seems incorrect, incorrect type of answer)
# question = "Is a marijuana dispensery allowed?"  # A: No
# question = "I would like to build a marijuana dispensery.  Which zoning districts permits this use?"  # A: []

answer = sem_par.classify(question)

print(f"Question: {question}")
if answer is True:
    print(f"Answer: Yes")
elif answer is False:
    print(f"Answer: No")
else:
    print(f"Answer: {answer}")

2022-12-02 18:26:47.681 | INFO     | question_classification:load_model:292 - Loading XGBoost model: question_classification_model.ubj


[[ 83570  85230  85717 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[2]
template name: template_use_1var_yn_answer
SPARQL TEMPLATE: 
ASK {
        ?zoning :permitsUse "${use}" .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(1.7109150355375984, 'quarries')]
SLOTS: {'use': 'quarries'}
SPARQL:  
ASK {
        ?zoning :permitsUse "quarries" .
}

====== The answer is ======
Yes
Question: Is mining allowed?
Answer: Yes
CPU times: user 1.29 s, sys: 87.9 ms, total: 1.37 s
Wall time: 1.82 s


# Metrics

Here are metrics. How well does it do.

In [2]:
%%time
# These take 44 minutes to run on CPU / 40 seconds on GPI
semantic_parsing.measure_accuracy()


!curl -d "Metrics are done 3" ntfy.sh/JeusixafHaygto

2022-12-02 18:38:00 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-02 18:38:01 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-02 18:38:01 INFO: Use device: cpu
2022-12-02 18:38:01 INFO: Loading: tokenize
2022-12-02 18:38:01 INFO: Loading: pos
2022-12-02 18:38:01 INFO: Loading: lemma
2022-12-02 18:38:01 INFO: Loading: depparse
2022-12-02 18:38:01 INFO: Done loading processors!


question_corpus_filt: 900


2022-12-02 18:38:04.274 | INFO     | question_classification:load_model:292 - Loading XGBoost model: question_classification_model.ubj


[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]


...

2022-12-02 19:38:34.713 | INFO     | question_classification:load_model:292 - Loading XGBoost model: question_classification_model.ubj


template name: template_use_2var_yn_answer
SPARQL TEMPLATE: 
ASK {
    ?zoning :permitsUse "${use}" ;
            rdfs:label "${zoning}" .
}

VARIABLES: ('use', 'zoning')
RELATIONS: None
SLOTS: {}
num_entity_slots: 2
SIMILARITY SCORES for num_entity_slots: [(1.5830679671955457, 'FI3'), (1.488949302917355, 'salt works')]
SLOTS FORWARD: {'use': 'FI3', 'zoning': 'salt works'}
SLOTS REVERSED: {'use': 'salt works', 'zoning': 'FI3'}
SPARQL:  
ASK {
    ?zoning :permitsUse "FI3" ;
            rdfs:label "salt works" .
}

====== The answer is ======
SPARQL:  
ASK {
    ?zoning :permitsUse "salt works" ;
            rdfs:label "FI3" .
}

====== The answer is ======
Yes
[[ 83569  33906  85230  33690  34359  33908  33906  85398  85717 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[3]


2022-12-02 19:38:37.808 | INFO     | question_classification:load_model:292 - Loading XGBoost model: question_classification_model.ubj


template name: template_use_2var_yn_answer
SPARQL TEMPLATE: 
ASK {
    ?zoning :permitsUse "${use}" ;
            rdfs:label "${zoning}" .
}

VARIABLES: ('use', 'zoning')
RELATIONS: None
SLOTS: {}
num_entity_slots: 2
SIMILARITY SCORES for num_entity_slots: [(2.369624270126813, 'petroleum refining'), (1.2260458465489485, 'FI3')]
SLOTS FORWARD: {'use': 'petroleum refining', 'zoning': 'FI3'}
SLOTS REVERSED: {'use': 'FI3', 'zoning': 'petroleum refining'}
SPARQL:  
ASK {
    ?zoning :permitsUse "petroleum refining" ;
            rdfs:label "FI3" .
}

====== The answer is ======
SPARQL:  
ASK {
    ?zoning :permitsUse "FI3" ;
            rdfs:label "petroleum refining" .
}

====== The answer is ======
Yes
[[ 83569  33906  85230  33690  34359  33908  33906  85398  85717 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

# Training

Training is a very fast process due to doing a simple fit.  There is not cross validation done on the data.

In [4]:
%%time
# takes apx. 2-3 minutes CPU time
# on first time run Stanza has to download some packages
import semantic_parsing
semantic_parsing.train_all()

2022-12-02 18:26:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-02 18:26:57 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-02 18:26:57 INFO: Use device: cpu
2022-12-02 18:26:57 INFO: Loading: tokenize
2022-12-02 18:26:57 INFO: Loading: pos
2022-12-02 18:26:58 INFO: Loading: lemma
2022-12-02 18:26:58 INFO: Loading: depparse
2022-12-02 18:26:58 INFO: Done loading processors!
2022-12-02 18:27:00.442 | INFO     | question_classification:train2:212 - 1. Question Classifier training begun. ================
2022-12-02 18:29:33.531 | INFO     | question_classification:train2:260 - Training XGBoost Classifier took: 153.08687949180603 s


Number of Questions Encoded: 900
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

/home/mdcochra/.conda/envs/NLP-SPARQL/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['A1', 'A2', 'C1', 'C2', 'C3', 'C4', 'FI1', 'FI2', 'FI3', 'R1', 'R1a', 'R1b', 'R1c', 'R1d', 'R2', 'R2a', 'R2b', 'R3', 'R3a', 'R3b', 'agricultural Use', 'alcohol manufacturing', 'amusement centers (including bowling alleys, golf driving ranges miniature golf courses, ice rinks, pool and billiard halls, and similar recreational uses)', 'auto body shops', 'auto-dismantling yards', 'automotive motor fuel dispensing facilities', 'automotive sales', 'automotive self-service motor fuel dispensing facilities', 'boat storage yards', 'breweries', 'building material supply sales (wholesale and retail)', 'buildings accessory to the permitted uses (including private garages and accessory living quarters)', 'business services', 'carpet manufacturing', 'commercial bakeries', 'commercial printing and publishing', 'community commercial centers (including wholesale and re

Iteration 1, loss = 6.18212010
Iteration 2, loss = 5.75491528
Iteration 3, loss = 5.35123558
Iteration 4, loss = 4.92551195
Iteration 5, loss = 4.47249138
Iteration 6, loss = 3.99515810
Iteration 7, loss = 3.51291825
Iteration 8, loss = 3.04351804
Iteration 9, loss = 2.60574319
Iteration 10, loss = 2.21216264
Iteration 11, loss = 1.87454666
Iteration 12, loss = 1.59365966
Iteration 13, loss = 1.36734841
Iteration 14, loss = 1.18819279
Iteration 15, loss = 1.04839393
Iteration 16, loss = 0.94228951
Iteration 17, loss = 0.85933561
Iteration 18, loss = 0.79569421
Iteration 19, loss = 0.74691966
Iteration 20, loss = 0.70725123
Iteration 21, loss = 0.67483479
Iteration 22, loss = 0.64844646
Iteration 23, loss = 0.62592906
Iteration 24, loss = 0.60688772
Iteration 25, loss = 0.58949243
Iteration 26, loss = 0.57444315
Iteration 27, loss = 0.56066825
Iteration 28, loss = 0.54770007
Iteration 29, loss = 0.53557606
Iteration 30, loss = 0.52438414
Iteration 31, loss = 0.51380693
Iteration 32, los

Iteration 261, loss = 0.04214789
Iteration 262, loss = 0.04175405
Iteration 263, loss = 0.04132986
Iteration 264, loss = 0.04095240
Iteration 265, loss = 0.04050200
Iteration 266, loss = 0.04013275
Iteration 267, loss = 0.03972858
Iteration 268, loss = 0.03931965
Iteration 269, loss = 0.03891921
Iteration 270, loss = 0.03854390
Iteration 271, loss = 0.03816836
Iteration 272, loss = 0.03779884
Iteration 273, loss = 0.03740965
Iteration 274, loss = 0.03705948
Iteration 275, loss = 0.03668373
Iteration 276, loss = 0.03633886
Iteration 277, loss = 0.03600310
Iteration 278, loss = 0.03565940
Iteration 279, loss = 0.03531719
Iteration 280, loss = 0.03499259
Iteration 281, loss = 0.03467451
Iteration 282, loss = 0.03431967
Iteration 283, loss = 0.03403128
Iteration 284, loss = 0.03365363
Iteration 285, loss = 0.03337415
Iteration 286, loss = 0.03302963
Iteration 287, loss = 0.03272766
Iteration 288, loss = 0.03238968
Iteration 289, loss = 0.03208215
Iteration 290, loss = 0.03180048
Iteration 